In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import folium

In [2]:
# https://github.com/vuski/admdongkor 참조. 여기서 데이터 받아왔음.

df = pd.read_csv('coordinate_UTMK_이름포함.tsv', sep='\t')
df.head()

,SERIAL,ADMCD,ADMNM,X,Y
0,0,1100000000,서울특별시,954919,1950195
1,1,1111000000,서울특별시 종로구,953003,1955642
2,2,1111051000,서울특별시 종로구 청운동,953064,1954062
3,3,1111051500,서울특별시 종로구 청운효자동,953064,1954062
4,4,1111052000,서울특별시 종로구 효자동,953064,1954062


In [3]:
df[df['ADMNM']=='경기도 수원시 장안구'].index #1608. 수원시 시작 idx
df[df['ADMNM']=='경기도 성남시 수정구'].index # 1690. 수원시 이후 시작 idx. 이건 그냥 노가다로 진행.
df_suwon = df.loc[1608:1690-1, ['ADMNM', 'X', 'Y', 'ADMCD']]
df_suwon.head()

,ADMNM,X,Y,ADMCD
1608,경기도 수원시 장안구,956043,1924450,4111100000
1609,경기도 수원시 장안구 신안동,957033,1920272,4111153000
1610,경기도 수원시 장안구 화서1동,955819,1920262,4111154000
1611,경기도 수원시 장안구 화서2동,954907,1920578,4111155000
1612,경기도 수원시 장안구 파장동,954636,1924765,4111156000


In [4]:
# 인구 data 받기. 출처 : 수원시청 홈페이지

df_population = pd.read_excel("2023년6월말기준.xls")
df_population.head()

,구,구·동,세대수,인구_합계,인구_내국인_인구수_계,인구_내국인_인구수_남,인구_내국인_인구수_여,인구_내국인_전월말인구수,인구_내국인_인구증감전월대비,외국인_계,외국인_남,외국인_여
0,장안구,파장동,10578,22899,22531,11400,11131,22900,-1,368,181,187
1,장안구,율천동,19805,43049,41011,21654,19357,43202,-153,2038,1187,851
2,장안구,정자1동,10828,28164,27934,13715,14219,28221,-57,230,106,124
3,장안구,정자2동,15128,36765,36302,17798,18504,36859,-94,463,215,248
4,장안구,정자3동,14174,40346,40157,20043,20114,40392,-46,189,88,101


In [5]:
# 필요한 data만 정리

df1 = df_population.loc[0:, ['구', '구·동', '인구_합계']]

df1["시군구"] = df1["구"] + " " + df1["구·동"] # 시군구 이름 정리

# 좌표 행 추가.
df1['X'] = 0
df1['Y'] = 0
df1['ADMCD'] = 0
for i in range(0, df1.shape[0]):
    row = df1.iloc[i]["시군구"]
    x_idx = df_suwon[df_suwon['ADMNM'].str.contains(row)]["X"].head(1)
    y_idx = df_suwon[df_suwon['ADMNM'].str.contains(row)]["Y"].head(1)
    admcd = df_suwon[df_suwon['ADMNM'].str.contains(row)]["ADMCD"].head(1)
    
    df1.iat[i, 2] = int(df1.iat[i,2]) # 2021행(인구)가 string이라 int로 바꿈
    df1.iat[i, 4] = int(x_idx.values)
    df1.iat[i, 5] = int(y_idx.values)
    df1.iat[i, 6] = int(admcd.values)
df1.head()   #shape(44, 6)

,구,구·동,인구_합계,시군구,X,Y,ADMCD
0,장안구,파장동,22899,장안구 파장동,954636,1924765,4111156000
1,장안구,율천동,43049,장안구 율천동,952896,1922591,4111156600
2,장안구,정자1동,28164,장안구 정자1동,954629,1923101,4111157100
3,장안구,정자2동,36765,장안구 정자2동,955482,1921579,4111157200
4,장안구,정자3동,40346,장안구 정자3동,954637,1922219,4111157300


In [8]:
df_suwon_idx = df1.loc[0:, ['X', 'Y']] #(44,2). 클러스터링을 위해 좌표만 추출.

# 20대 선거구 획정시 단위 선거구당 최대/최소 인구. 이것은 이미 설정된 것
maxPeopleNum = 310053   
minPeopleNum = 103351

model = KMeans(n_clusters = 5, n_init=10)
ansList = []
ansClusteredList=[]
num_failure = 0

# range 횟수를 늘려가며 클러스터링 진행. 다양한 클러스터링 case 중에서 인구 조건을 만족하는 것을 중첩되지 않게 저장
# range 횟수가 증가할수록 도출되는 클러스터 case 개수가 일정 값으로 수렴하기 때문에 무작정 range 횟수를 반복하지 않았음
# 100 -> 500 -> 1000 -> ... 해서 ansList의 크기가 일정값에 수렴한다고 판단되면 클러스터링 과정 완료함
for i in range(100) :
    model.fit(df_suwon_idx)
    df1["cluster"] = model.labels_
    
    tempList=[]
    for j in range(5) :
        tempSum = df1[df1['cluster']==j]["인구_합계"].sum()
        if not (tempSum>= minPeopleNum and tempSum<= maxPeopleNum) : 
            num_failure +=1
            break
        else : tempList.append(tempSum)
            
        if(j==4) :
            tempList.sort()
            if tempList not in ansList:
                ansList.append(tempList)
                ansClusteredList.append(model.labels_)
            

#df_suwon[df_suwon['cluster']==0]
#sns.scatterplot(x="X", y="Y", hue="cluster", data=df1, palette="Set2")

In [9]:
#차례대로 클러스터링 진행한 결과 각각의 선거구별 인구 수, 도출된 가능한 선거구 획정의 case, 실패한 횟수(인구 조건 미충족. 단순 확인용)

for i in range(len(ansList)) :
    print(ansList[i])
print(len(ansList))
print(num_failure)

[213254, 221921, 231945, 259422, 300771]
[221921, 231945, 232826, 239006, 301615]
[221921, 231945, 232826, 259422, 281199]
[214110, 221921, 232826, 279139, 279317]
[171061, 221921, 231945, 300771, 301615]
[213254, 221921, 236946, 275875, 279317]
[214110, 221921, 231945, 257722, 301615]
[221921, 232826, 239006, 254243, 279317]
[212050, 213254, 221921, 279317, 300771]
[210969, 213254, 223002, 279317, 300771]
[221921, 231945, 238150, 259422, 275875]
[210969, 213254, 247898, 275875, 279317]
12
4


In [10]:
# 도출된 선거구 획정 case 각각마다, 수원시 시군구가 어느 선거구에 속하는지 0~4로 표시된 것
# ex : 2로 표시된 것은 같은 선거구로 묶인 시군구이다. 
# 시군구의 순서는 df1의 ['시군구'] 열 참조

for i in range(len(ansClusteredList)) :
    print(ansClusteredList[i])

[1 1 1 1 1 1 1 1 1 1 2 2 2 3 3 3 3 3 2 2 0 3 2 2 2 2 2 1 4 4 4 2 4 4 0 4 4
 4 4 0 0 0 0 0]
[1 4 1 1 1 1 1 1 1 1 0 0 0 0 4 4 4 4 0 0 2 4 0 0 0 0 0 4 3 3 3 0 3 3 2 3 3
 3 3 2 2 2 2 2]
[0 2 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 1 1 4 2 1 1 1 1 1 2 3 3 3 1 3 3 4 3 3
 3 3 4 4 4 4 4]
[1 3 1 1 1 1 1 1 1 1 0 0 0 0 3 3 3 3 0 4 4 3 0 0 0 0 0 0 2 2 2 0 2 2 4 2 2
 2 2 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 0 0 0 0 2 2 2 2 0 0 1 2 0 0 0 0 0 4 3 3 3 0 3 3 1 3 3
 3 3 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 2 2 2 4 4 4 4 4 2 3 3 4 2 2 2 2 2 2 0 0 0 2 0 0 3 0 0
 0 0 3 3 3 3 3]
[2 3 2 2 2 2 2 2 2 2 1 1 1 1 3 3 3 3 1 1 4 3 1 1 1 1 1 2 0 0 0 1 0 0 4 0 0
 0 0 4 4 4 4 4]
[4 1 4 4 4 4 4 4 4 4 2 2 2 2 1 1 1 1 2 3 3 1 2 2 2 2 2 1 0 0 0 2 0 0 3 0 0
 0 0 3 3 3 3 3]
[1 1 1 1 1 1 1 1 1 1 2 2 2 3 3 3 3 3 2 0 0 3 2 2 2 2 2 1 4 4 4 2 4 4 0 4 4
 4 4 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 3 3 3 4 4 4 4 4 3 2 2 4 3 3 3 3 3 1 3 0 0 3 0 0 2 0 0
 0 0 2 2 2 2 2]
[1 1 1 1 1 1 1 1 1 1 0 0 0 2 2 2 2 2 0 0 3 2 0 0 0 0 0 2 4 4 4 0 4 4 3 4 4
 4 4 3 3 3 3 3]

In [12]:
# ansClusteredList가 n인 경우의 시군구 확인 용도
df1[ansClusteredList[0]==4]

,구,구·동,인구_합계,시군구,X,Y,ADMCD,cluster
28,팔달구,지동,10952,팔달구 지동,957965,1920494,4111553000,1
29,팔달구,우만1동,19854,팔달구 우만1동,958630,1920898,4111554100,1
30,팔달구,우만2동,17347,팔달구 우만2동,959001,1920256,4111554200,1
32,영통구,매탄1동,13251,영통구 매탄1동,959236,1919173,4111751000,1
33,영통구,매탄2동,14605,영통구 매탄2동,960160,1919138,4111752000,1
35,영통구,매탄4동,21026,영통구 매탄4동,960094,1918673,4111754000,1
36,영통구,원천동,42027,영통구 원천동,961044,1919341,4111755000,1
37,영통구,광교1동,53584,영통구 광교1동,959880,1921347,4111760000,1
38,영통구,광교2동,29275,영통구 광교2동,961889,1921565,4111761000,1


In [13]:
# 지도로 시각화

import geopandas as gpd
# 좌표 시작점 : 서울 추정
latitude = 37.394946
longitude = 127.111104
geo_data = 'HangJeongDong_ver20230701.geojson' # https://github.com/vuski/admdongkor 참조

kor = gpd.read_file('HangJeongDong_ver20230701.geojson', encoding='cp949')

In [16]:
#수원시 지역 경계만 추출

suwon_idx = kor[kor['temp'].str.contains("수원시")] #44개
suwon_idx.head()

,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
1145,경기도 수원시장안구 파장동,3101154,4111156000,41111,41,경기도,수원시장안구 파장동,수원시장안구,31011540,"MULTIPOLYGON (((127.00612 37.33807, 127.00706 ..."
1146,경기도 수원시장안구 율천동,3101155,4111156600,41111,41,경기도,수원시장안구 율천동,수원시장안구,31011550,"MULTIPOLYGON (((126.97274 37.30552, 126.97304 ..."
1147,경기도 수원시장안구 정자1동,3101156,4111157100,41111,41,경기도,수원시장안구 정자1동,수원시장안구,31011560,"MULTIPOLYGON (((126.99709 37.30314, 126.99726 ..."
1148,경기도 수원시장안구 정자2동,3101157,4111157200,41111,41,경기도,수원시장안구 정자2동,수원시장안구,31011570,"MULTIPOLYGON (((127.00935 37.29537, 127.01096 ..."
1149,경기도 수원시장안구 영화동,3101158,4111158000,41111,41,경기도,수원시장안구 영화동,수원시장안구,31011580,"MULTIPOLYGON (((127.01761 37.28783, 127.01694 ..."


In [17]:
# 경계 지역 데이터(suwon_idx)와 인구 및 좌표 포함 데이터(df1) 지역 코드 통일

for i in range(0, df1.shape[0]):
    row = df1.iloc[i]["구·동"]
    admcd = suwon_idx[suwon_idx['adm_nm'].str.contains(row)]["adm_cd2"].head(1)
    df1.iat[i, 6] = int(admcd.values)
df1.head   #shape(44, 6)

<bound method NDFrame.head of       구   구·동  인구_합계       시군구       X        Y       ADMCD  cluster
0   장안구   파장동  22899   장안구 파장동  954636  1924765  4111156000        4
1   장안구   율천동  43049   장안구 율천동  952896  1922591  4111156600        2
2   장안구  정자1동  28164  장안구 정자1동  954629  1923101  4111157100        4
3   장안구  정자2동  36765  장안구 정자2동  955482  1921579  4111157200        4
4   장안구  정자3동  40346  장안구 정자3동  954637  1922219  4111157300        4
5   장안구   영화동  20817   장안구 영화동  956917  1921521  4111158000        4
6   장안구   송죽동  19142   장안구 송죽동  955941  1923236  4111159100        4
7   장안구  조원1동  29302  장안구 조원1동  957140  1922483  4111159700        4
8   장안구  조원2동  17948  장안구 조원2동  957030  1923447  4111159800        4
9   장안구   연무동  17443   장안구 연무동  957751  1924599  4111160000        4
10  권선구  세류1동  10910  권선구 세류1동  956196  1918079  4111352000        0
11  권선구  세류2동  25516  권선구 세류2동  956521  1916290  4111353000        0
12  권선구  세류3동  19336  권선구 세류3동  956849  1917981  4111354000        0
13  

In [18]:
# match 안 된 것 있는지 확인
adm_cd_to_int = []

for i in suwon_idx['adm_cd2'] :
    adm_cd_to_int.append(int(i))
    
    
for i in range(0, df1.shape[0]):
    if df1.iat[i,6] not in adm_cd_to_int :
        print(df1.iat[i,3])



In [20]:
# 지도로 시각화. 색 구분

m = folium.Map(location=[37.2782037, 126.9793158],  
                    zoom_start=11)
# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=suwon_idx,
    data=df1,
    columns=('ADMCD', 'cluster'),
    key_on='feature.properties.adm_cd2',
    bins=5,
    fill_color='BrBG',
    legend_name='선거구',
    nan_fill_opacity=0.5,
).add_to(m)

# 맵 m을 출력합니다.
m